In [2]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [22]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import xgboost as xgb
import numpy as np


In [25]:

def datacoll(df_2y, df_10y, df_rec):

    df_2y['Date'] = pd.to_datetime(df_2y['Date'])
    df_2y.set_index('Date', inplace=True)


    df_10y['Date'] = pd.to_datetime(df_10y['Date'])
    df_10y.set_index('Date', inplace=True)

    df_2y.drop(columns=[' Open', ' High', ' Low'], inplace=True)
    df_10y.drop(columns=[' Open', ' High', ' Low'], inplace=True)

    df_rec['Date'] = pd.to_datetime(df_rec['Date'])


    date_range = pd.date_range(start=df_rec['Date'].min(), end=df_rec['Date'].max())
    df_rec = df_rec.set_index('Date').reindex(date_range)

    df_rec['val'] = df_rec['val'].ffill()

    df_rec = df_rec.reset_index().rename(columns={'index': 'Date'})
    df_rec.set_index('Date', inplace=True)

    df_2_10 = pd.merge(df_2y, df_10y, left_index=True, right_index=True, how='inner')
    df_2_10.columns = df_2_10.columns.str.strip()

    df_2_10['recession'] = (df_2_10['Close_y'] - df_2_10['Close_x']) < 0
    df_2_10['recession'] = df_2_10['recession'].astype(int)

    df_2_10.index = pd.to_datetime(df_2_10.index)
    df_2_10.index = df_2_10.index + pd.DateOffset(years=1)
    df_2_10.index = df_2_10.index.strftime('%Y-%m-%d')
    df_2_10.index = pd.to_datetime(df_2_10.index)

    df = pd.merge(df_2_10, df_rec, left_index=True, right_index=True, how='inner')

    return df


In [27]:
def do(bond, df):
  correlation = df['recession'].corr(df['val'])

  print(bond, 'correlation:' , correlation)

  X = df[['val']]
  y = df['recession']

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

  model = LogisticRegression()
  model.fit(X_train, y_train)

  y_pred = model.predict(X_test)

  accuracy = accuracy_score(y_test, y_pred)
  print(f'Logistic Regression Accuracy: {accuracy}')

  rf_model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
  rf_model.fit(X_train, y_train)

  rf_y_pred = rf_model.predict(X_test)

  rf_accuracy = accuracy_score(y_test, rf_y_pred)
  print(f'Random Forest Accuracy: {rf_accuracy}')

  X = df[['val']].values
  y = df['recession'].values

  encoder = LabelEncoder()
  y = encoder.fit_transform(y)

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

  model = Sequential([
      Dense(8, input_dim=1, activation='relu'),
      Dense(1, activation='sigmoid')
  ])

  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

  model.fit(X_train, y_train, epochs=50, batch_size=2, verbose=0)

  loss, accuracy = model.evaluate(X_test, y_test)
  print(f'Neural Network Accuracy: {accuracy}')






# **India**

In [28]:
df_2 = pd.read_csv(r'/content/drive/My Drive/ind2year.csv')
df_10 = pd.read_csv(r'/content/drive/My Drive/ind10year.csv')
df_rec = pd.read_csv(r'/content/drive/My Drive/indrecession.csv')
x=datacoll(df_2, df_10, df_rec)
x.drop

<ipython-input-25-39a49dfb1a4f>:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_2y['Date'] = pd.to_datetime(df_2y['Date'])
<ipython-input-25-39a49dfb1a4f>:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_10y['Date'] = pd.to_datetime(df_10y['Date'])


<bound method DataFrame.drop of             Close_x  Close_y  recession  val
Date                                        
2022-01-08    4.141    5.893          0  0.0
2022-01-07    4.197    5.893          0  0.0
2022-01-06    4.133    5.898          0  0.0
2022-01-05    4.115    5.856          0  0.0
2022-01-04    4.118    5.864          0  0.0
...             ...      ...        ...  ...
2011-01-18    6.242    7.708          0  1.0
2011-01-15    6.277    7.736          0  1.0
2011-01-14    6.280    7.700          0  1.0
2011-01-13    6.328    7.663          0  1.0
2011-01-12    6.353    7.800          0  1.0

[2813 rows x 4 columns]>

In [29]:
do('India', x)

India correlation: -0.12027284128795614
Logistic Regression Accuracy: 0.9158767772511849
Random Forest Accuracy: 0.9158767772511849


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9094 - loss: 0.2942  
Neural Network Accuracy: 0.9158768057823181


# **U.S.**

In [30]:
df_2 = pd.read_csv(r'/content/drive/My Drive/us2year.csv')
df_10 = pd.read_csv(r'/content/drive/My Drive/us10year.csv')
df_rec = pd.read_csv(r'/content/drive/My Drive/usrecession.csv')
x=datacoll(df_2, df_10, df_rec)

<ipython-input-25-39a49dfb1a4f>:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_2y['Date'] = pd.to_datetime(df_2y['Date'])
<ipython-input-25-39a49dfb1a4f>:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_10y['Date'] = pd.to_datetime(df_10y['Date'])


In [31]:
do('U.S.', x)

U.S. correlation: 0.4981569696856843
Logistic Regression Accuracy: 0.9112654320987654
Random Forest Accuracy: 0.9112654320987654


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9065 - loss: 0.2280  
Neural Network Accuracy: 0.9112654328346252


# **U.K.**

In [32]:
df_2 = pd.read_csv(r'/content/drive/My Drive/uk2year.csv')
df_10 = pd.read_csv(r'/content/drive/My Drive/uk10year.csv')
df_rec = pd.read_csv(r'/content/drive/My Drive/ukrecession.csv')
x=datacoll(df_2, df_10, df_rec)

<ipython-input-25-39a49dfb1a4f>:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_2y['Date'] = pd.to_datetime(df_2y['Date'])
<ipython-input-25-39a49dfb1a4f>:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_10y['Date'] = pd.to_datetime(df_10y['Date'])
<ipython-input-25-39a49dfb1a4f>:13: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_rec['Date'] = pd.to_datetime(df_rec['Date'])


In [33]:
do('U.K.', x)

U.K. correlation: 0.3609121044283354
Logistic Regression Accuracy: 0.8835672997522709
Random Forest Accuracy: 0.8835672997522709


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8985 - loss: 0.2907  
Neural Network Accuracy: 0.8835672736167908


# **Germany**

In [34]:
df_2 = pd.read_csv(r'/content/drive/My Drive/ger2year.csv')
df_10 = pd.read_csv(r'/content/drive/My Drive/ger10year.csv')
df_rec = pd.read_csv(r'/content/drive/My Drive/germanyrecession.csv')
x=datacoll(df_2, df_10, df_rec)

<ipython-input-25-39a49dfb1a4f>:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_2y['Date'] = pd.to_datetime(df_2y['Date'])
<ipython-input-25-39a49dfb1a4f>:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_10y['Date'] = pd.to_datetime(df_10y['Date'])


In [35]:
do('Germany', x)

Germany correlation: -0.05469044452182939
Logistic Regression Accuracy: 0.9948096885813149
Random Forest Accuracy: 0.9948096885813149


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9963 - loss: 0.0239
Neural Network Accuracy: 0.9948096871376038


# **France**

In [36]:
df_2 = pd.read_csv(r'/content/drive/My Drive/fr2year.csv')
df_10 = pd.read_csv(r'/content/drive/My Drive/fr10year.csv')
df_rec = pd.read_csv(r'/content/drive/My Drive/francerecession.csv')
x=datacoll(df_2, df_10, df_rec)

<ipython-input-25-39a49dfb1a4f>:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_2y['Date'] = pd.to_datetime(df_2y['Date'])
<ipython-input-25-39a49dfb1a4f>:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_10y['Date'] = pd.to_datetime(df_10y['Date'])


In [37]:
do('France', x)

France correlation: -0.021548645130397236
Logistic Regression Accuracy: 0.999137187230371
Random Forest Accuracy: 0.999137187230371


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9993 - loss: 0.0056      
Neural Network Accuracy: 0.9991371631622314
